# Bootstrap Your Own Latent: A New Approach to Self-Supervised Learning

- paper review

## Abstract  
- BYOL은 2개의 network를 사용해 각각 online, target이라 부르며 이들은 서로 상호작용하며 학습한다  
- 하나의 이미지를 augment해서 online network를 훈련시켜 다른 augment view와 동일한 image의 target network representation을 예측한다  
- 그리고 동시에 online network의 slow-moving average를 가지고 target network를 업데이트한다  
- BYOL은 ImageNet classification task에서 74.3%의 accuracy를 보여준다  
- backbone으로 larger ResNet을 사용할 경우, 79.6%까지 올라간며 SSL에서 SOTA를 달성한다

## Introduction  
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/BYOL/figures/figure1.png?raw=true" width=400>  


- contrastive method는 positive pair의 distance는 줄이고 negative pair의 distance는 증가시키면서 학습한다  
- 이러한 method들은 large batch size, memory bank 또는 customized mining에 의존해 negative pair을 세심하게 처리해야 한다  
- 우리가 제안하는 BYOL은 negative pair 없이 contrastive method의 SOTA보다 좋은 성능을 보여준다  
- BYOL은 향상된 representation을 위해 network의 output을 반복적으로 bootstrap하며 target으로 serve한다  
- 또한 BYOL은 augmentation 선택에 있어 다른 method들보다 robust하다  
- 우리는 negative pair에 의존하지 않는 이유 중 하나가 BYOL의 robustness라고 생각하고 있다  
- 하나의 이미지에 2개의 augment view를 얻으면 BYOL은 같은 이미지에서 다른 하나의 view의 target representation을 학습한다  
- 모든 이미지에서 같은 vector가 나오는 것처럼 collapsed을 허용하지만 BYOL은 실험적으로 이러한 결과에 수렴하지 않는다  
- online network에 predictor를 추가하고 online parameter의 slow-moving average를 target network로 사용하는 것은 online projection 내에서 더 많은 정보를 encoding하도록 하며 collapsed problem을 피한다고 우리는 가설을 세운다  

## Method  
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/BYOL/figures/figure2.png?raw=true" width=600>  


- 일반적으로 contrastive method는 다른 augmented view를 예측함으로써 image의 representation을 학습한다  
- 그래서 하나의 augmented view에서 다른 하나의 augmented view를 예측한다  
- 그러나 representation space에서 바로 예측을 하는 것은 항상 같은 representation을 출력하는 것처럼 collapsed representation으로 유도할 수 있다   
- contrastive method는 이 문제를 discrimination 문제로 재구성해 collapsed problem을 해결한다  
- 하나의 view representation이 주어지면 다른 view representation과 구분하기 위해 학습한다  
- 그러나 discriminative approach는 각 representation 비교를 위해 많은 negative sample을 필요로 한다  
- 이 연구에서 우리는 collapse problem을 피하기 위해 negative sample이 필수적인지 알아보았다  
- 간단한 solution으로 먼저 random initialized network를 고정시키는 것이다  
- 이는 실험적으로 낮은 성능 1.4%의 정확도를 보여준다  
- 그러나 우리의 method를 적용하면 18.8%의 정확도를 얻는다  
- 이것이 BYOL의 중요한 motivation이다  
- target representation이 주어지면 우리는 이를 예측함으로써 새로운 online representation을 학습할 수 있다  
- 이로부터 우리는 위의 procedure를 반복하면 representation의 퀄리티가 증가한다는 것을 예상할 수 있다  
- 실제로 BYOL은 이를 통해 일반화된 성능을 얻으며 이때 slow-moving average를 사용해야 한다  

### Description of BYOL  
- BYOL의 목표는 downstream task에서 쓸 수 있는 representation $y_{\theta}$를 학습하는 것이다  
- online network는 weight $\theta$의 set으로 정의되고 encoder $f_{\theta}$, projector $g_{\theta}$, predictor $q_{\theta}$ 3개의 stage로 구성된다  
- target network도 같은 architecture를 가지지만 다른 weight $\xi$로 업데이트 되며 online network를 학습하기 위한 regression target을 제공한다  
- $\xi$는 exponential moving average로 업데이트 되며 식은 다음과 같다  
$$\xi \leftarrow \tau\xi + \left(1-\tau\right)\theta$$  
- BYOL의 pipeline을 거치면 onlinedms $g_{\theta}\left(z_{\theta}\right)$를 출력하고 $g_{\theta}\left(z_{\theta}\right)$와 $z'_{\xi}$를 각각 nomalized한다  
- 이때 predictor는 online network에만 적용이 되므로 두 architecture는 비대칭이다  
- 마지막으로 MSE를 정의하는데 normalized된 prediction과 target projection의 error를 구한다  
- 각 step마다 loss를 optimize하는 데 이때 $\theta$만 업데이트를 하고 $\xi$는 업데이트를 하지 않는다  
$$\theta \leftarrow optimizer\left(\theta, \nabla_{\theta}\mathcal{L}^{BYOL}_{\theta,\xi}, \eta\right)$$  
$$\xi \leftarrow \tau\xi + \left(1-\tau\right)\theta$$  

- BYOL의 pipeline을 거치면 online은 $q_{\theta}\left(z_{\theta}\right)$를 출력하고 $q_{\theta}\left(z_{\theta}\right)$와 $z'_{\xi}$를 각각 normalize한다  
- 이때 predictorsms online network에만 적용이 되므로 두 architecture는 비대칭이다  
- 마지막으로 MSE를 정의하는데 normalized된 prediction과 target projection의 error를 구한다 
$$\mathcal{L}_{\theta, \xi}\overset{\underset{\mathrm{def}}{}}{=}\Vert\bar{q_{\theta}}\left(z_{\theta}\right)-\bar{z'_{\xi}}\Vert_2^2=2-2\cdot\frac{<q_{\theta}\left(z_{\theta}\right),z'_{\theta}>}{\Vert q_{\theta}\left(z_{\theta}\right)\Vert_2\cdot\Vert z'_{\xi}\Vert_2}$$
- 위의 loss function에서 2개의 view를 바꿔서도 propagation시켜 $\tilde{\mathcal{L}}_{\theta, \xi}$를 얻으며 최종 loss function은 다음과 같다  
$$\mathcal{L}^{BYOL}_{\theta,\xi}=\mathcal{L}_{\theta,\xi}+\tilde{\mathcal{L}}_{\theta,\xi}$$  

- 각 step마다 loss를 optimize하는데 이때 $\theta$만 업데이트하고 $\xi$는 업데이트를 하지 않는다  

### Intuitions on BYOL's behavior  
- GAN에서 generator와 discriminator의 loss를 같이 계산하는 term이 없듯이 BYOL도 마찬가지로 $\theta$와 $\xi$가 같이 업데이트 되지 않는다  
- 그래서 GAN에서와 비슷하게 loss가 수렴하지 않는 경우가 있다  
- BYOL의 dynamic은 undesirable equilibria를 야기하지만 저자들은 실험적으로 겪어보진 못했다  
- BYOL의 predictor의 optimal을 다음과 같이 정의하자  
$$q^{\star}\overset{\underset{\mathrm{def}}{}}{=}argmin_{q}E\left[\Vert q\left(z_{\theta}\right)-z'_{\xi}\Vert_2^2\right]$$  
- 그러면 다음과 같이 전개할 수 있다  
$$\nabla_{\theta}E\left[\Vert q^{\star}\left(z_{\theta}\right)-z'_{\xi} \Vert_2^2\right]=\nabla_{\theta}E\left[\Vert E\left[z'_{\xi}\vert z_{\theta}\right] -z'_{\xi}\Vert_2^2\right]=\nabla_{\theta}E\left[\sum_i Var\left(z'_{\xi,i}\vert z_{\theta}\right)\right]$$  
- 따라서 expected conditional variance의 gradient로 업데이트를 한다  
- 임의의 random variable X, Y, Z에 대해 $Var\left(X \vert Y, Z\right)\le Var\left(X \vert Y\right)$가 성립한다  
- target projection을 X, current online projection을 Y, Z는 predictor와 같은 additional variable이라 가정하자  
- 이때 Z의 information을 완전히 버린다고 해서 conditional variance를 줄일 순 없다  
- 임의의 cosntant $c$와 random variable $z_{\theta}$에 대해 $Var\left(z'_{xi}\vert z_{\theta}\right)\ge Var\left(z'_{\xi}\vert c\right)$이므로 BYOL이 수렴하게 돼도 expected conditional variance가 커서 불안정하다  
- 따라서 collapsed problem을 해결할 수 있다  
- 대신 $\xi$와 $\theta$를 가깝게 만들어 online projection에서 캡처된 다양성을 target projection에 반영한다  
- 이런 식으로 $\xi$의 업데이틀르 진행한다  
- 그러나 parameter가 갑작스럽게 바뀌는 경우 online network의 optimal predictor의 assumption이 깨질 수 있다  
- 그래서 $\xi$는 $\theta$의 moving average를 통해 천천히 업데이트 되도록 한다  

### Implementation details  
**Image Augmentation**  
- resized to $224\times 224$  
- random horizontal flip  
- color distortion consiting of a random squence of brightness, cotrast, saturation, hue adjustments  
- grayscale  
- gaussian blur  
- solarization  

**Architecture**  
- We use convolutional residual network (ResNet) as our base parametric encoders $f_{\theta}$ and $f_{\xi}$  
- The dimension of final output layer after average pooling : 2048  
- SimCLR처럼 representation $y$를 MLP $g_{\theta}$와 연결해준다  
- 4096 node의 linear layer, batch normalization, relu를 연결하고 256 dimension의 output을 출력한다  

**Optimization**  
- LARS optimizer  
- cosien decay scheduler without restarts  
- 1000 epochs  
- 10 warmup epoch  
- base learning rate to 0.2 scaled linearly with the batch size (lr = 0.2 x batch size/256) 
- weight decay of $1.5\cdot 10^{-6}$  
- moving average paramter $\tau$ starts from $\tau_{base}=0.996$  
- we set $\tau=1-\left(1-\tau_{base}\right)\cdot\left(cos\left(\pi k / K\right)+1\right)/2$, where K is the maximum number of training steps

## Experimental evaluation  
**Linear evaluation on ImageNet**  
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/BYOL/figures/table1.png?raw=true">  

**Semi-supervised training on ImageNet**  
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/BYOL/figures/table2.png?raw=true">  

**Transfer to other classification tasks**  
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/BYOL/figures/table3.png?raw=true">  

**Transfer to other vision tasks**  
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/BYOL/figures/table4.png?raw=true">

## Building intuitions with ablations  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/BYOL/figures/figure3.png?raw=true">  

**Batch size**  
- contrastive method들에서 negative sample 때문에 batch size에 따라 성능에 큰 영향을 미치는 경우가 많다  
- 그러나 BYOL은 robust한 성능을 보여준다   

**Image augmentations**  
- contrastive method들에서 augmentation에 대해 매우 민감한 경우가 많다  
- 예를 들어 SimCLR에서는 color distortion을 지우고 학습시키면 그 성능이 현저히 떨어진다  
- 반면 BYOL은 역시 robust한 성능을 보여준다  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/BYOL/figures/table5.png?raw=true">

**Bootstrapping**  
$$InfoNCE^{\alpha, \beta}_{\theta}=\frac{2}{B}\sum_{i=1}^B S_{\theta}\left(v_i, v'_i\right)-\beta\cdot \frac{2\alpha}{B}\sum_{i=1}^B ln\left(\sum_{j\neq i} exp \frac{S_{\theta}\left(v_i,v_j\right)}{\alpha}+\sum_j exp\frac{S_{\theta}\left(v_i,v'_j\right)}{\alpha}\right)$$  
- $\beta=1$이 일반적인 InfoNCE이고 $\beta=0$가 BYOL의 loss function이다  
- table 5(b)를 통해 negative sample에 의존하지 않으면서 가장 좋은 representation을 얻는 variant는 $\beta=0$이면서 predictor와 target을 모두 사용한 BYOL이란 것을 알 수 있다  

## Conclusion  
- BYOL은 negative pair을 사용하지 않으면서 output의 이전 버전을 예측해 representation을 학습한다  
- 우리는 BYOL이 vision의 각 task에서 SOTA를 달성했음을 보여준다  
- 그럼에도 불구하고 BYOL은 여전히 augmentation에 종속적이다  
- BYOL을 통해 다른 modalities로 확장하기 위해선 각 data에 맞게 augmentation을 할 필요가 있으며 여기에는 해당 전문지식을 필요로 한다  
- BYOL이 modalities로 확장할 수 있는 step이 되었으면 좋겠다